# Import packages

In [32]:
import os
import sys
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from xarrayMannKendall import Mann_Kendall_test
from dask.distributed import Client
from astropy import convolution
directory   = '/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS'
sys.path.append('/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS/Figures')
import PUG_functions as PF
rcParams['font.family'] = 'sans-serif'


# Read SSH data

In [33]:
dataset = xr.open_dataset(directory+'/AVISO/AVISO_monthly_SSH_LS_detrend.nc')
lon     = np.tile(dataset.lon, (len(dataset.lat),1)).transpose()
lat     = np.tile(dataset.lat, (len(dataset.lon),1))
large_scale_ssh1 = dataset.adt.transpose().values
large_scale_ssh2 = large_scale_ssh1.copy()
large_scale_ssh1[:,:30,:]=np.nan;
large_scale_ssh1[:,182:,:]=np.nan;
# large_scale_ssh1[:5,:,:]=np.nan;
large_scale_ssh2[:,:30,:]=np.nan;
large_scale_ssh2[:636,:,:]=np.nan;
large_scale_ssh2[905:,:,:]=np.nan;
###################################################################################################
dataset1   = sio.loadmat(directory+'/Figures/Figure4_lat_SG_WBC_detrend.mat')
lat_SG_KC_trend = dataset1['lat_SG_KC_trend'][:,0]
lat_SG_GS_trend = dataset1['lat_SG_GS_trend'][:,0]
lat_SG_KC_mean  = dataset1['lat_SG_KC_mean'][:,0]
lat_SG_GS_mean  = dataset1['lat_SG_GS_mean'][:,0]
lat_SG_mean      = np.array((lat_SG_KC_mean,lat_SG_GS_mean))
###################################################################################################
ds = xr.Dataset(data_vars=dict(lat_SG_mean=(["latitude","time"], lat_SG_mean)))
lat_SG_WBC_mean0 = ds.lat_SG_mean
lat_SG_WBC_mean1 = lat_SG_WBC_mean0.sortby('time')
lat_SG_WBC_mean  = Mann_Kendall_test(lat_SG_WBC_mean1,'time',MK_modified=True,method="linregress",alpha=0.05,
                                coords_name = {'time':'time','x':'latitude'})
lat_SG_WBC_mean_trend = lat_SG_WBC_mean.compute()
###################################################################################################
time_start = np.datetime64('1993-07')
time_end   = np.datetime64('2023-08')
t_range    = PF.time_range(time_start,time_end,'monthly','np64')

# Plot the Subtropical Ocean Gyres

In [ ]:
fig_ratio = 0.9
fig       = plt.figure(figsize=(20, 30))
gs        = gridspec.GridSpec(3,2)
labelfont     = 15
padspacescale = 10
labelpadscale = 2
linefont      = 1.0
scale1        = 1.5
scale2        = 0.65
labels1       = ['a','b','c','d','e']
cmaps1        = cmocean.cm.matter
##################################################################################
# domains of Subtropical Gyres
lon_SG_KC1 = np.array([135,170,170,135,135])
lon_SG_KC2 = np.array([141,153,153,141,141])
lat_SG_KC1 = np.array([30,30,40,40,30])
lat_SG_KC2 = np.array([30,30,40,40,30])
lon_SG_GS1 = np.array([285,315,315,285,285])
lon_SG_GS2 = np.array([285,295,295,285,285])
lat_SG_GS1 = np.array([30,30,42,42,30])
lat_SG_GS2 = np.array([30,30,40,40,30])
############################### Row 1 ###################################
ax = fig.add_subplot(gs[0:1])
l, b, w, h = ax.get_position().bounds
m  = Basemap(projection='merc',suppress_ticks=True,llcrnrlat=np.nanmin(lat),urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='c')        
m.drawmeridians(np.arange(0,360, 20),labels=[0,0,0,1],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')
m.drawparallels(np.arange(0,55, 10),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif') 
m.drawcoastlines(color='0.1',  linewidth=0.3*linefont)
m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
x, y  = m(lon, lat)
cx,cy = m(101, 52)
ax.set_position([l+0.005, b, scale1*w, scale1*h])   
ax.spines['left'].set_linewidth(labelpadscale)
ax.spines['top'].set_linewidth(labelpadscale)
ax.spines['right'].set_linewidth(labelpadscale)
ax.spines['bottom'].set_linewidth(labelpadscale)
for i in range(2):
    if i==0:
        SSH=large_scale_ssh1
    else:
        SSH=large_scale_ssh2       
    for j in range(np.size(large_scale_ssh1,2)):
        if i<1:
            CS=m.contour(x, y,SSH[:,:,j],levels=[1.1],linewidths=0.1*linefont,linestyles='dashed',colors='tab:gray')
        else:
            CS=m.contour(x, y,SSH[:,:,j],levels=[0.25],linewidths=0.1*linefont,linestyles='dashed',colors='tab:blue')    
x_SG_KC1,y_SG_KC1=m(lon_SG_KC1,lat_SG_KC1)
x_SG_GS1,y_SG_GS1=m(lon_SG_GS1,lat_SG_GS1)
m.plot(x_SG_KC1,y_SG_KC1,linewidth=2*linefont,linestyle='solid', color='tab:gray')
m.plot(x_SG_GS1,y_SG_GS1,linewidth=2*linefont,linestyle='solid', color='tab:blue')
m.fillcontinents(color='0.9', lake_color='white')
plt.xlabel('Longitude',fontsize=labelfont,labelpad=3*padspacescale,family='sans-serif')
plt.ylabel('Latitude',fontsize=labelfont,labelpad=5*padspacescale,family='sans-serif')
ax.axes.xaxis.set_tick_params(pad=0)
ax.axes.yaxis.set_tick_params(pad=0)
plt.text(cx, cy, labels1[0],color='xkcd:black', fontsize=labelfont, weight='bold',family='sans-serif')
############################### Row 2 ###################################
for i in range(2):
    ax = fig.add_subplot(gs[i+2])
    l, b, w, h = ax.get_position().bounds
    b = b + 0.21
    if i==0:
        l = l+0.0
        SSH   = large_scale_ssh1
        m     = Basemap(projection='merc',llcrnrlat=25-0.05,urcrnrlat= 45+0.05,llcrnrlon=135,urcrnrlon=155,resolution='l')
        cx,cy = m(130, 46.0)
        x_SG_KC2,y_SG_KC2 = m(lon_SG_KC2,lat_SG_KC2)
        m.plot(x_SG_KC2,y_SG_KC2,linewidth=2*linefont,linestyle='solid', color='tab:gray')
    else:
        l = l-0.13
        SSH   = large_scale_ssh2
        m     = Basemap(projection='merc',llcrnrlat=25-0.05,urcrnrlat= 45+0.05,llcrnrlon=280,urcrnrlon=300,resolution='l')    
        cx,cy = m(275, 46.0)
        x_SG_GS2,y_SG_GS2=m(lon_SG_GS2,lat_SG_GS2)
        m.plot(x_SG_GS2,y_SG_GS2,linewidth=2*linefont,linestyle='solid', color='tab:blue')
    m.drawmeridians(np.arange(0,360, 5),labels=[0,0,0,1],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')
    m.drawparallels(np.arange(15,50, 5),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')    
    m.drawcoastlines(color='0.1',  linewidth=0.3*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    x, y = m(lon, lat)
    for j in range(np.size(large_scale_ssh1,2)):
        if i==0:
            CS=m.contour(x, y,SSH[:,:,j], levels=[1.1], linewidths=0.1*linefont,linestyles='dashed',colors='tab:gray')
        else:
            CS=m.contour(x, y,SSH[:,:,j], levels=[0.25], linewidths=0.1*linefont,linestyles='dashed',colors='tab:blue')    
    m.fillcontinents(color='0.9', lake_color='white')
    ax.set_position([l, b, scale2*w, scale2*h])   
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    ax.spines['bottom'].set_linewidth(labelpadscale)
    plt.text(cx, cy,labels1[i+1],color='xkcd:black', fontsize=labelfont, weight='bold')
    plt.ylabel('Latitude',fontsize=labelfont,labelpad=5*padspacescale,family='sans-serif')
    plt.xlabel('Longitude',fontsize=labelfont,labelpad=2.5*padspacescale,family='sans-serif')
    ax.axes.xaxis.set_tick_params(pad=0)
    ax.axes.yaxis.set_tick_params(pad=0)
############################### Row 3 ###################################
for i in range(2):
    ax = fig.add_subplot(gs[i+4]) 
    l, b, w, h = ax.get_position().bounds
    b = b+0.32
    if i<1:
        l=l+0.02
        plt.plot(t_range, lat_SG_KC_mean, color='black', linewidth=2*linefont)
        plt.plot(t_range, lat_SG_KC_trend, color='black', linewidth=2*linefont,linestyle='dashed')
        plt.text(t_range[0]-85, 36.3, labels1[i+3],fontsize = labelfont, color='black', weight='bold')
        plt.title('Pacific Ocean', fontsize=labelfont, loc='left',pad=padspacescale,family='sans-serif')
    else:
        l=l-0.12
        plt.plot(t_range, lat_SG_GS_mean,  color='black', linewidth=2*linefont)
        plt.plot(t_range, lat_SG_GS_trend, color='black', linewidth=2*linefont,linestyle='dashed')
        plt.text(t_range[0]-50, 38.7, labels1[i+3],fontsize = labelfont, color='black', weight='bold')
        plt.title('Atlantic Ocean', fontsize=labelfont, loc='left',pad=padspacescale,family='sans-serif')
    ax.set_position([l, b-0.01, 0.6*w, 0.6*h])
    plt.xlim(t_range[0], t_range[-1])
    xminorLocator   = mpl.dates.YearLocator(2)
    xmajorLocator   = mpl.dates.YearLocator(4)
    xmajorFormatter = mpl.dates.DateFormatter('%Y')
    ax.xaxis.set_minor_locator(xminorLocator)
    ax.xaxis.set_major_locator(xmajorLocator)
    ax.xaxis.set_major_formatter(xmajorFormatter)
    plt.xticks(fontsize=labelfont,rotation=45)
    ax.axes.xaxis.set_tick_params(pad=labelpadscale*0.5)
    plt.xlabel('Year',fontsize=labelfont,labelpad=2,family='sans-serif')
    if i<1:
        plt.ylim(33, 36)
        yminorLocator   = plt.MultipleLocator(0.1)
        ymajorLocator   = plt.MultipleLocator(0.2)        
        plt.plot([t_range[20], t_range[70]], [35.75,35.75], color='black', linewidth=2*linefont)
        plt.plot([t_range[20], t_range[70]], [35.5,35.5], color='black', linewidth=2*linefont,linestyle='dashed')
        plt.text(t_range[75], 35.7125, 'Mean',  fontsize = labelfont, color='black',family='sans-serif')
        plt.text(t_range[75], 35.4625, 'Trend', fontsize = labelfont, color='black',family='sans-serif')
        plt.ylabel(r'Latitude($^\circ$)', fontsize=labelfont, labelpad=0,family='sans-serif')
        ax.set_yticks(np.linspace(33, 36, 7))
        ax.set_yticklabels(['33$^\circ$S','33.5$^\circ$S','34$^\circ$S','34.5$^\circ$S','35$^\circ$S','35.5$^\circ$S','36$^\circ$S'])
    else:
        yminorLocator   = plt.MultipleLocator(1)
        ymajorLocator   = plt.MultipleLocator(1)
        plt.ylim(36.5, 38.5)  
        plt.plot([t_range[20], t_range[70]], [38.3,38.3], color='black', linewidth=2*linefont)
        plt.plot([t_range[20], t_range[70]], [38.1,38.1], color='black', linewidth=2*linefont,linestyle='dashed')
        plt.text(t_range[75], 38.275, 'Mean',  fontsize = labelfont, color='black',family='sans-serif')
        plt.text(t_range[75], 38.075, 'Trend', fontsize = labelfont, color='black',family='sans-serif')
        ax.set_yticks(np.linspace(36.5, 38.5, 5))
        ax.set_yticklabels(['36.5$^\circ$S','37$^\circ$S','37.5$^\circ$S','38$^\circ$S','38.5$^\circ$S'])
    plt.yticks(fontsize=labelfont,family='sans-serif')
    ax.axes.yaxis.set_tick_params(pad=labelpadscale)
    ax.spines['bottom'].set_linewidth(labelpadscale)
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    plt.tick_params(axis='x',which='minor',bottom='on',left='on',top='on',right='on',\
                length=5,width=2,colors='black',direction='in')
    plt.tick_params(axis='y',which='minor',bottom='on',left='on',top='on',right='on',\
                length=5,width=2,colors='black',direction='in')
    plt.tick_params(bottom='on',left='on',top='on',right='on',length=8.0,width=2,colors='black',direction='in')
fig.savefig(directory+'/Figures/Figure4_ssh_contour.png',dpi=300,bbox_inches = 'tight')